In [0]:
#General imports
import glob
import os
import numpy as np
import pandas as pd
from xml.etree import ElementTree as ET

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

#!ls "/content/gdrive/My Drive"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Get home directory
# HOME = os.environ['HOME']

# Set the directory where we saved the corpus of fake news spreaders
DIREC = '/content/gdrive/My Drive/Language_Processing_2/langpro2_project/pan20-author-profiling-training-2020-02-23/'

# Set the language
LANG  = "en/"
# LANG = "es/"

In [0]:
# Getting X and y
GT = DIREC+LANG+"/truth.txt"
true_values = {}
f=open(GT)
for line in f:
    linev = line.strip().split(":::")
    true_values[linev[0]] = linev[1]
f.close()

X = []
y = []
ID = []

for FILE in glob.glob(DIREC+LANG+"*.xml"):
    # The split command below gets just the file name,
    # without the whole address. The last slicing part [:-4]
    # removes .xml from the name, so that to get the user code
    
    USERCODE = FILE.split("/")[-1][:-4]
    parsedtree = ET.parse (FILE)
    documents  = parsedtree.iter("document")
    
    ID.append(USERCODE)
    # This function should return a vectorial representation of a user
    # repr = get_representation_tweets (FILE)
    
    # We append the representation of the user to the X variable
    # and the class to the y vector
    # X.append(repr)

    texts =[]
    for doc in documents:
        texts.append(doc.text) #original text
        #texts.append(prepare_text_for_lda(doc.text)) # tokenize and clean texts
        # y.append(int(true_values[USERCODE]))
        # X.append(doc.text)

    X.append(" ".join(texts))
    # x.append(" ".join(texts).split()) # append a cleaned joined bag-of-words list to a collective x
    y.append(int(true_values[USERCODE]))

In [0]:
X = np.array(X)
y = np.array(y)

In [0]:
# tutorial link: https://www.youtube.com/watch?v=Y_hzMnRXjhI&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=3
import tensorflow as tf
from tensorflow import keras

print("TensorFlow version:", tf.__version__)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

TensorFlow version: 2.2.0


In [0]:
training_size = 200
vocab_size = 6000
embedding_dim = 64
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
num_epochs = 500
verbosity = 1

In [0]:
def keras_tokenizer(XTrain, XTest):
    tokenizer = Tokenizer(num_words=vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, 
                      split=' ', char_level=False, oov_token=oov_tok)
    
    tokenizer.fit_on_texts(XTrain)
    word_index = tokenizer.word_index
    
    XTrain_seq = tokenizer.texts_to_sequences(XTrain)
    XTrain_padded = pad_sequences(XTrain_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    
    XTest_seq = tokenizer.texts_to_sequences(XTest)
    XTest_padded = pad_sequences(XTest_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    XTrain_padded = np.array(XTrain_padded)
    XTest_padded = np.array(XTest_padded)

    return XTrain_padded, XTest_padded 

In [0]:
from sklearn.model_selection import StratifiedKFold

# Define the K-fold Cross Validator
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=False)

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# K-fold Cross Validation model evaluation
fold_no = 1

for train_index, test_index in skf.split(X, y):
    
    XTrain, XTest = X[train_index], X[test_index]
    yTrain, yTest = y[train_index], y[test_index]
    
    XTrain = XTrain.tolist()
    XTest = XTest.tolist()

    train_len = []
    for item in XTrain:
        train_len.append(len(item.split()))
    train_len = np.array(train_len)
    max_length = np.ndarray.max(train_len)
    print(fold_no, "max_length:", max_length)

    XTrain_padded, XTest_padded = keras_tokenizer(XTrain, XTest)
    
    # Define the model architecture
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Conv1D(64, kernel_size=3, strides=1),
    tf.keras.layers.MaxPooling1D(pool_size=30),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
    
    # Compile the model
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    # Fit data to model
    history = model.fit(XTrain_padded, yTrain, epochs=num_epochs, validation_data=(XTest_padded, yTest), verbose=verbosity)
    
    # Generate generalization metrics
    scores = model.evaluate(XTest_padded, yTest, verbose=0)
    #print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    # Increase fold number
    fold_no = fold_no + 1

Streaming output truncated to the last 5000 lines.
Epoch 10/500
8/8 [==============================] - 0s 28ms/step - loss: 0.0551 - accuracy: 0.9875 - val_loss: 1.1253 - val_accuracy: 0.6833
Epoch 11/500
8/8 [==============================] - 0s 28ms/step - loss: 0.0289 - accuracy: 0.9917 - val_loss: 1.4278 - val_accuracy: 0.6667
Epoch 12/500
8/8 [==============================] - 0s 30ms/step - loss: 0.0170 - accuracy: 0.9958 - val_loss: 1.2915 - val_accuracy: 0.6167
Epoch 13/500
8/8 [==============================] - 0s 28ms/step - loss: 0.0127 - accuracy: 0.9917 - val_loss: 1.1364 - val_accuracy: 0.6667
Epoch 14/500
8/8 [==============================] - 0s 28ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.9021 - val_accuracy: 0.6667
Epoch 15/500
8/8 [==============================] - 0s 30ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 1.0112 - val_accuracy: 0.6667
Epoch 16/500
8/8 [==============================] - 0s 28ms/step - loss: 0.0016 - accuracy: 1.0000 - va

In [0]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 4.715627193450928 - Accuracy: 69.9999988079071%
------------------------------------------------------------------------
> Fold 2 - Loss: 1.447269082069397 - Accuracy: 66.66666865348816%
------------------------------------------------------------------------
> Fold 3 - Loss: 5.037782669067383 - Accuracy: 68.33333373069763%
------------------------------------------------------------------------
> Fold 4 - Loss: 4.602751731872559 - Accuracy: 66.66666865348816%
------------------------------------------------------------------------
> Fold 5 - Loss: 5.085775852203369 - Accuracy: 66.66666865348816%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 67.66666769981384 (+- 1.3333320617675781)
> Loss: 4.177841305732727
-----------------------------------

In [0]:
#import matplotlib.pyplot as plt


#def plot_graphs(history, string):
  #plt.plot(history.history[string])
  #plt.plot(history.history['val_'+string])
  #plt.xlabel("Epochs")
  #plt.ylabel(string)
  #plt.legend([string, 'val_'+string])
  #plt.show()
  
#plot_graphs(history, "accuracy")
#plot_graphs(history, "loss")